# 关于Patch Merging问题的讨论
patch merging操作的目的是将feature map进行下采样并且通道维度翻倍，我在重复代码的时候使用的是kernel size为2，步长为2的卷积实现此操作，而swin transformer中官方代码在patch merging过程中使用了一系列操作，将水平和竖直方向像素块间隔一个拼接再重新堆叠（实现了水平和竖直尺寸缩减为二分之一，通道数增加了四倍），然后再做个layer norm，之后通道维度做线性层降低一半的的维度。整个过程见下面两段代码演示，假设下面生成的是三通道4\*4的feature map转变成的高2宽2通道为3\*4的feature map

In [2]:
import torch
from torch import nn

import matplotlib.pyplot as plt
%matplotlib inline

x1 = torch.ones(size=(4,4,3)).int()
x1[1::2, 0::2, 0] = 255
x1[0::2, 1::2, 1] = 255
x1[1::2, 1::2, 2] = 255
ax = plt.subplot(1, 2, 1)
ax.imshow(x1)
ax.set_title("before")
ax.set_xticks([])
ax.set_yticks([])

In [3]:
x2 = x1.view(2,2,2,2,3).transpose(1,2).reshape(2,2,12)
for i in range(4):
    ax = plt.subplot(1, 4, i+1)
    ax.imshow(x2[:,:,3*i:3*(i+1)])
    ax.set_title(f"ch{i}")
    ax.set_xticks([])
    ax.set_yticks([])

仔细思考后发现其实做卷积操作和拼接后再做线性映射（或者1*1卷积）没什么不同，FLOPs经过计算也是相同的，无非初始化的方式不同有一定程度的影响，不明白为何采用这么麻烦的拼接再做线性映射，后来明白这种对像素的间隔取样是种常规操作虽然FLOPs相同但计算起来这种线性映射更快，根据朱毅老师在某评论区提出的观点看应该也是提高效率的一种技巧，并指出使用pixel shuffle对效率提升可能更有效果，之前没听说过pixel shuffle，但根据名字猜想应该跟shuffleNet中的channel shuffle是差不多的东西，实现上来说就是对应的维度经过转置再重新reshape成符合的维度，直接省略掉了索引加拼接的步骤，简单的测试如下。

- 首先验证一下两种方法得到的结果是相同的

In [8]:
def shuffle_pix(x):
    H, W, C = x.shape
    return x.view(H//2,2,W//2,2,C).permute(0,2,1,3,4).reshape(H//2,W//2,4*C)

def split_pix(x):
    H, W, C = x.shape
    x0 = x[0::2, 0::2, :]  # B H/2 W/2 C
    x1 = x[1::2, 0::2, :]  # B H/2 W/2 C
    x2 = x[0::2, 1::2, :]  # B H/2 W/2 C
    x3 = x[1::2, 1::2, :]  # B H/2 W/2 C
    x = torch.cat([x0, x2, x1, x3], -1)  # B H/2 W/2 4*C
    return x.view(H//2, W//2, -1)  # B H/2*W/2 4*C

x = torch.randn(56,56,96)
y1 = shuffle_pix(x)
y2 = split_pix(x)
(y1==y2).all()

- 根据swin transformer官方代码修改，将pixel shuffle操作做简单的不严谨的对比。（之前在gpu上官方patch merging比卷积快五倍左右，shuffle操作只比官方patch merging操作快了一点点）。

In [5]:
class PatchMerging(nn.Module):
    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm, pix_shuffle=True):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)
        self.pix_shuffle= pix_shuffle

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."
        
        x = x.view(B, H, W, C)
        
        if self.pix_shuffle:
            x = x.view(B,H//2,2,W//2,2,C).permute(0,1,3,2,4,5).reshape(B,H//2,W//2,4*C)
        else:
            x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
            x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
            x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
            x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
            x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
            x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x
    
import time
x = torch.randn(128,56*56,96)
patch_merge = PatchMerging((56,56),96,pix_shuffle=False)
shuffle_pix = PatchMerging((56,56),96)
t1 = time.time()
patch_merge(x)
t_end = time.time()-t1
print(f"patch merging用时{t_end}")

In [6]:
t1 = time.time()
shuffle_pix(x)
t_end = time.time()-t1
print(f"pix shuffle用时{t_end}")